## Optimization Solver 

In [1]:
import gurobipy as gp
from gurobipy import GRB
from pyepo.model.grb import optGrbModel

class myModel(optGrbModel):
    def _getModel(self):
        """
        A method to build Gurobi model

        Returns:
            tuple: optimization model and variables
        """
        # ceate a model
        m = gp.Model("Simple Model")
        # varibles
        x = m.addVars([1,2], name="x", vtype=GRB.BINARY)
        # sense
        m.modelSense = GRB.MAXIMIZE
        # constraints
        m.addConstr(2 * x[1] + 2 * x[2] <= 3)
        return m, x

Auto-Sklearn cannot be imported.


In [2]:
# test
optmodel = myModel()
optmodel.setObj([0,1])
sol, obj = optmodel.solve()
print("Obj: {}".format(obj))
print("x_1 = {:.2f}, x_2 = {:.2f}".format(sol[0], sol[1]))

Set parameter Username
Academic license - for non-commercial use only - expires 2024-01-01
Obj: 1.0
x_1 = -0.00, x_2 = 1.00


## Data Set

In [3]:
import time

from gurobipy import GRB
import numpy as np
import torch
from torch.utils.data import Dataset
from tqdm import tqdm

from pyepo.model.opt import optModel


class optDatasetConstrs(Dataset):
    """
    This class is Torch Dataset for optimization problems with active constraints.

    Attributes:
        model (optModel): Optimization models
        feats (np.ndarray): Data features
        costs (np.ndarray): Cost vectors
        sols (np.ndarray): Optimal solutions
        objs (np.ndarray): Optimal objective values
        ctrs (list(np.ndarray)): active constraints
    """

    def __init__(self, model, feats, costs):
        """
        A method to create a optDataset from optModel

        Args:
            model (optModel): an instance of optModel
            feats (np.ndarray): data features
            costs (np.ndarray): costs of objective function
        """
        if not isinstance(model, optModel):
            raise TypeError("arg model is not an optModel")
        self.model = model
        # data
        self.feats = feats
        self.costs = costs
        # find optimal solutions
        self.sols, self.objs, self.ctrs = self._getSols()

    def _getSols(self):
        """
        A method to get optimal solutions for all cost vectors
        """
        sols, objs, ctrs = [], [], []
        print("Optimizing for optDataset...")
        time.sleep(1)
        for c in tqdm(self.costs):
            try:
                sol, obj = self._solve(c)
                constrs = self._getBindingConstrs()
            except:
                raise ValueError(
                    "For optModel, the method 'solve' should return solution vector and objective value."
                )
            sols.append(sol)
            objs.append([obj])
            ctrs.append(np.array(constrs))
        return np.array(sols), np.array(objs), ctrs

    def _solve(self, cost):
        """
        A method to solve optimization problem to get an optimal solution with given cost

        Args:
            cost (np.ndarray): cost of objective function

        Returns:
            tuple: optimal solution (np.ndarray) and objective value (float)
        """
        self.model.setObj(cost)
        sol, obj = self.model.solve()
        return sol, obj
    
    def _getBindingConstrs(self):
        """
        A method to get active constraints with current optimal solution

        Returns:
            np.ndarray: normal vector of constraints
        """
        constrs = []
        # iterate all constraints
        for constr in self.model._model.getConstrs():
            # check tight constraints
            if abs(constr.Slack) < 1e-5:
                t_constr = []
                # get coefficients
                for i in self.model.x:
                    t_constr.append(self.model._model.getCoeff(constr, self.model.x[i]))
                # get coefficients in standard form
                if constr.sense == GRB.LESS_EQUAL:
                    # <=
                    constrs.append(t_constr)
                elif constr.sense == GRB.GREATER_EQUAL:
                    # >=
                    constrs.append([- coef for coef in t_constr])
                elif constr.sense == GRB.EQUAL:
                    # ==
                    constrs.append(t_constr)
                    constrs.append([- coef for coef in t_constr])
                else:
                    # invalid sense
                    raise ValueError("Invalid constraint sense.")
        # iterate all constraints
        for i, v in enumerate(self.model._model.getVars()):
            t_constr = [0] * len(self.model.x)
            # check variables on bounds
            if v.x <= 1e-5:
                # x_i >= 0
                t_constr[i] = -1
                constrs.append(t_constr)
            elif v.x >= 1 - 1e-5:
                # x_i <= 1
                t_constr[i] = 1
                constrs.append(t_constr)
        return constrs
                
    def __len__(self):
        """
        A method to get data size

        Returns:
            int: the number of optimization problems
        """
        return len(self.costs)

    def __getitem__(self, index):
        """
        A method to retrieve data

        Args:
            index (int): data index

        Returns:
            tuple: data features (torch.tensor), costs (torch.tensor), optimal solutions (torch.tensor) and objective values (torch.tensor)
        """
        return (
            torch.FloatTensor(self.feats[index]),
            torch.FloatTensor(self.costs[index]),
            torch.FloatTensor(self.sols[index]),
            torch.FloatTensor(self.objs[index]),
            torch.FloatTensor(self.ctrs[index])
        )

In [4]:
import pyepo

In [5]:
# generate data from shortest path
grid = (2,2) # grid size
num_data = 1000 # number of training data
num_feat = 5 # size of feature
deg = 4 # polynomial degree
e = 0.5 # noise width
feats, costs = pyepo.data.shortestpath.genData(num_data+1000, num_feat, grid, deg, e, seed=42)

In [6]:
# keep 2 vars
costs = costs[:,:2]

In [7]:
# split data
from sklearn.model_selection import train_test_split
x_train, x_test, c_train, c_test = train_test_split(feats, costs, test_size=1000, random_state=42)
# get training and test data set
dataset_train = optDatasetConstrs(optmodel, x_train, c_train)
dataset_test = optDatasetConstrs(optmodel, x_test, c_test)

Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4660.66it/s]


Optimizing for optDataset...


100%|████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:00<00:00, 4803.18it/s]


In [8]:
# get data loader
from torch.utils.data import DataLoader
batch_size = 32
loader_train = DataLoader(dataset_train, batch_size=batch_size, shuffle=True)
loader_test = DataLoader(dataset_test, batch_size=batch_size, shuffle=False)

## Prediction Model

In [9]:
import torch
from torch import nn

# build linear model
class LinearRegression(nn.Module):
    
    def __init__(self):
        super(LinearRegression, self).__init__()
        self.linear = nn.Linear(num_feat, 2)
    
    def forward(self, x):
        out = self.linear(x)
        return out

In [10]:
# init model
reg = LinearRegression()

## Loss 

In [11]:
from torch.autograd import Function
from torch.nn import functional as F
from pyepo import EPO

class polarConeAngle(nn.Module):
    """
    A autograd module for polar cone fitting loss with binary variables
    """
    
    def __init__(self, optmodel):
        """
        Args:
            optmodel (optModel): an PyEPO optimization model
        """
        super().__init__()
        # optimization model
        if not isinstance(optmodel, optModel):
            raise TypeError("arg model is not an optModel")
        self.optmodel = optmodel
    
    def forward(self, pred_cost, tight_ctrs):
        """
        Forward pass
        """
        loss = polarConeAngleFunc.apply(pred_cost, tight_ctrs, self.optmodel)
        return loss

    
class polarConeAngleFunc(Function):
    """
    A autograd function for polar cone fitting loss with binary variables
    """
    
    @staticmethod
    def forward(ctx, pred_cost, tight_ctrs, optmodel):
        # get device
        device = pred_cost.device
        # get batch size
        batch_size = len(pred_cost)
        # init loss
        loss = torch.empty(batch_size).to(device)
        # constraints to numpy
        tight_ctrs = tight_ctrs.cpu().detach().numpy()
        for i in range(batch_size):
            # get projection
            if optmodel.modelSense == EPO.MINIMIZE:
                # minimize
                p = getProjection(-pred_cost[i], tight_ctrs[i])
            if optmodel.modelSense == EPO.MAXIMIZE:
                # maximize
                p = getProjection(pred_cost[i], tight_ctrs[i])
            # calculate cosine similarity
            loss[i] = - F.cosine_similarity(F.normalize(pred_cost[i].unsqueeze(0)), 
                                            F.normalize(p.unsqueeze(0)))
            print("Predicted Cost:")
            print(pred_cost[i].detach().numpy())
            print("Projection:")
            print(p.detach().numpy())
            print("Loss:")
            print(loss[i].item())
            break
        return loss

        
def getProjection(cp, ctr):
    """
    A function to get the projection of the vector onto the polar cone via solving a quadratic programming
    """
    # ceate a model
    m = gp.Model("shortest path")
    # turn off output
    m.Params.outputFlag = 0
    # varibles
    p = m.addVars(len(cp), name="x", lb=-GRB.INFINITY, ub=GRB.INFINITY)
    λ = m.addVars(len(ctr), name="lambda")
    # onjective function
    obj = gp.quicksum((cp[i].item() - p[i]) ** 2 for i in range(len(cp)))
    m.setObjective(obj, GRB.MINIMIZE)
    # constraints
    print("Binding Constraints:")
    for constr in ctr:
        print(constr)
    for i in range(len(cp)):
        m.addConstr(gp.quicksum(ctr[j,i] * λ[j] for j in range(len(ctr))) == p[i])
    # focus on numeric problem
    m.Params.NumericFocus = 3
    # solve
    m.update()
    m.optimize()
    # get solutions
    λ_val = np.array([λ[i].x for i in λ])
    # normalize
    λ_norm = λ_val / np.linalg.norm(λ_val)
    # get normalized projection
    proj = torch.FloatTensor(λ_norm@ctr)
    return proj

In [12]:
# init loss
pca_loss = polarConeAngle(optmodel)

## Test 

In [13]:
for data in loader_train:
    x, c, w, z, t_ctr = data
    # forward pass
    cp = reg(x)
    loss = pca_loss(cp, t_ctr)
    # check sol
    for i in range(len(cp)):
        print("Cost:")
        print(c[i].detach().numpy())
        optmodel.setObj(cp[i])
        wpi, zpi = optmodel.solve()
        print("Solution:")
        print("True:", w[i].detach().numpy(), "Prediction:", wpi)
        print("True: {:.2f}, Prediction: {:.2f}".format(z[i].item(), c[i].detach().numpy()@wpi))
        break
    break

Binding Constraints:
[-1.  0.]
[0. 1.]
Predicted Cost:
[1.0932196  0.43150395]
Projection:
[0. 1.]
Loss:
-0.367144376039505
Cost:
[0.14596194 0.5064134 ]
Solution:
True: [0. 1.] Prediction: [1.0, 0.0]
True: 0.51, Prediction: 0.15
